[Haskell implementation](http://www.danielbrice.net/blog/10/)

[Doubly Recursive Multivariate Automatic Differentiation](http://www1.american.edu/cas/mathstat/People/kalman/pdffiles/mmgautodiff.pdf)

In [8]:
-- define Dual numbers to be a pair of numbers
data Dual = Dual  {x :: Float, dx :: Float}  deriving (Show,Eq)
-- overload operators/define algebra
instance Num Dual where -- notation? dx or x'
    (Dual x x') + (Dual y y') = Dual (x + y) (x' + y') -- sum rule
    (Dual x x') - (Dual y y') = Dual (x - y) (x' - y') 
    (Dual x x') * (Dual y y') = Dual (x * y) (x * y' + x' * y) -- product rule
instance Fractional Dual where
    (Dual x x') / (Dual y y') = Dual (x / y) ((y * y' - x * x') / y*y) -- quotient rule
    fromRational x = Dual (fromRational x) 0
-- test
let f x1 x2 = x1*x2 in f(Dual 4 1)(Dual 2 0)

Dual {x = 8.0, dx = 2.0}

In [9]:
-- overload elementary functions 
instance Floating Dual where 
    sin (Dual x x') = Dual (sin x) (x' * cos x) 
    cos (Dual x x') = Dual (cos x) (x' * (-(sin x))) 
    exp (Dual x x') = Dual (exp x) (x' * exp x)
-- test
let f x1 x2 = x2 * sin((x1 * x1) + 3.0) in f(Dual 5 1)(Dual 2 0) 

Dual {x = 0.5418116, dx = -19.252117}

Cool, how about for a scalar variable with higher order derivatives? Want $(f,f',f'',f''', ... , f^n)$

In [10]:
data Dual = FirstOrder {x::Float, d::Float} | HigherOrder Float Dual deriving (Show,Eq)
HigherOrder 4 (HigherOrder 2 (FirstOrder 1 4))

HigherOrder 4.0 (HigherOrder 2.0 (FirstOrder {x = 1.0, d = 4.0}))